In [1]:
__author__ = "Brian Arbuckle, Benu Atri and Paras Jamil"
__version__ = "CS224u, Final Project 2021 v1"
__updated__ = "Mar 23, 2021, 11pmCST"

In [2]:
import os
import logging
from lxml import etree
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime as dt

from parse_token_utils import create_directory, parse_fileName_vals
from filing_parsers import *

today = str(dt.today()).split(' ')[0]
today

'2021-03-24'

<div class="alert alert-info">
    <b>At some point</b><br> We will need to define what the labels are '1's and '0's as the discrepancy between performance_earnings and performance_filing this is true for about 10% - this is related to our hypothesis, the text that we need to show that it changes sentiment

In [3]:
## copied & adapted from data-engineering/parsers/filing_parsers.py (git)
forms = ['10-K']
def quick_parse(filepath):
    parser = etree.XMLParser(recover=True, huge_tree=True)
    tree = etree.parse(filepath, parser)
    notags = etree.tostring(tree, encoding='utf8', method='text')
    return notags
def parse_10q(filepath, cik, year):
    """
    Parses the 10-Q passed. Returns a dictionary:
        body: The body of the 10-Q
    :param filepath:
    :return:
    """
    from bs4 import BeautifulSoup
    results = {}
    with open(filepath) as fp:
        soup = BeautifulSoup(fp, "lxml")

    #results['TenKText'] = __extract_document_text(soup)
    results['headers'] = __parse_sec_header(soup, cik, year)
    results['documents'] = __parse_documents(soup)
    return results

def get_date(filingPath):
    """From FinBert"""
    accession_code = filingPath.split('/')[-2]
    ticker = filingPath.split('/')[-4]
    filing_type = filingPath.split('/')[-3]
    #print(accession_code, ticker, filing_type)
    code = accession_code+'.txt'
    with open(filingPath, 'r') as f:
        readlin = f.read()
    for i in readlin.split('\n'):
#         print(i)
        if code in i:
            date_extr = i.split(' : ')[1]
            return date_extr


def __parse_documents(soup):
    import time
    # Get all the douments
    result_documents = []
    documents = soup.find_all('document')

    for doc in documents:

        type_node = doc.find('type')
        if type_node:
            type_text = type_node.contents[0].strip()
            desc_node = type_node.find('description')
        else:
            type_text = 'NA'

        seq_node = doc.find('sequence')
        if seq_node:
            seq_text = seq_node.contents[0].strip()
        else:
            seq_text = str(int(time.time()))[-6:]

        if desc_node:
            desc_text = desc_node.contents[0]
        else:
            desc_text = ''

        if type_text in forms:
            is_10_q = True
        else:
            is_10_q = False

        if type_text not in ["XML", "GRAPHIC", "EXCEL", "ZIP"]:
            logging.debug("Parsing doucment type: {}".format(type_text))
            result_documents.append(
                {'is_10_q': is_10_q,
                 'type': type_text,
                 'sequence': seq_text,
                 'description': desc_text,
                 'document': __extract_document_text(soup)}
            )

    return result_documents

def __parse_sec_header(soup, cik, year):
    sec_header = soup.find("sec-header")

    if not sec_header:
        sec_header = soup

    result = {
        'cik': cik,
        'year': year,
    }
    result['accession_number'] = __get_line_item(sec_header, 'ACCESSION NUMBER:')
    result['conformed_period_of_report'] = __get_line_item(sec_header, 'CONFORMED PERIOD OF REPORT:')
    result['filed_as_of_date'] = __get_line_item(sec_header, 'FILED AS OF DATE:')
    result['company_confirmed_name'] = __get_line_item(sec_header, 'COMPANY CONFORMED NAME:')
    result['central_index_key'] = __get_line_item(sec_header, 'CENTRAL INDEX KEY:')
    result['standard_industrial_classification'] = __get_line_item(sec_header, 'STANDARD INDUSTRIAL CLASSIFICATION:')
    result['state_of_incorporation'] = __get_line_item(sec_header, 'STATE OF INCORPORATION:')
    result['fiscal_year_end'] = __get_line_item(sec_header, 'FISCAL QUARTER END:')

    return result

def __get_line_item(sec_header, attr_name):
    find_results = re.findall(attr_name + '(.*?)\n', str(sec_header))

    if find_results:
        return find_results[0].strip()
    else:
        return None
def __extract_document_text(soup):
    if len(soup) == 1:
        TenKtext = soup.find('text').extract().text
        TenKtext = parser_custom(TenKtext)
    #tables = document.find_all('table')
    #for table in tables:
    #    table.decompose()

    return TenKtext

In [ ]:
dataset_

In [4]:
FULL_DATASET = True
PICKLES = '../pickle_data/'
DATA = 'data'

# GLOBAL START AND END:
START = '2016-01-01'
END = '2021-03-01'

doc_folder_name = 'item7_parsed_documents'

if FULL_DATASET == True:
    stocks_PATH = os.path.join(PICKLES, "performance_labeled.pickle")
forms = [ '10-K']

In [5]:
stocks_PATH

'../pickle_data/performance_labeled.pickle'

In [6]:
stocks_df = pd.read_pickle(stocks_PATH)

In [7]:
stocks_df.head()
#parsed_sec_filings_path = f'../data/{today}_parsed_sec_filings_path/'
#create_directory(parsed_sec_filings_path) 
parsed_sec_filings_path = '../data/2021-03-20_parsed_sec_filings_path/'

In [ ]:
# uncomment below - if we needed to re-download, 

forms = ['10-K']
from bs4 import BeautifulSoup
#create_directory(os.path.join(sec_filings_path,doc_folder_name)) #if exists, will not create again
sec_filings_path = '../data/sec-edgar-filings/'
def run_parse(stocks_df):
    num_stocks = stocks_df.shape[0]
    c = 0

    for idx, row in stocks_df.iterrows():
        if row['form'] in forms:
                
            ticker_form_path = os.path.join(sec_filings_path, row['ticker'], row['form'][:4])
            ciks = os.listdir(ticker_form_path)
            #print(ticker_form_path)
            for cik in ciks:  
                #print(cik)
                if not cik == '.DS_Store':
                    text_to_parse = os.path.join(ticker_form_path, cik, 'full-submission.txt')
                    if row['date_filed'] == dt.strptime(get_date(text_to_parse), '%Y%m%d'):
                        c += 1
                        print("{:0>2d} of {} {}".format(c, num_stocks, (30*'-')))
                        ticker, cik_10dig, filing_type, filing_qrtr, doc_reported_on, doc_year = parse_fileName_vals(cik, row, text_to_parse)
                        #input()
                        
                        out_file_name = ticker+'_'+cik_10dig+'_'+filing_type+'_'+filing_qrtr+ ".txt"
                        out_file_path = os.path.join(parsed_sec_filings_path, out_file_name)
                        
                        if os.path.exists(out_file_path):
                            continue
                            
                        print(f'Parsing a {filing_type} for {ticker}, quarter {filing_qrtr}, dated {doc_reported_on}...')
                        #results = parse_10q(text_to_parse, cik, doc_year)
                        
    
                        with open(text_to_parse) as fp:
                            soup = BeautifulSoup(fp, "lxml")
                        tenK = __extract_document_text(soup)
                        if tenK:
                            print(len(tenK))
                            #print(len(results))
                            with open(out_file_path, 'w') as f:
                                f.write(tenK)
                                print()
                            print(f'Parsing finished, output saved in:\n{out_file_path}')
                            print('__'*30)
                            print()
                                                    
                        else:
                            print('Bad command or file name', text_to_parse)                        
run_parse(stocks_df)

01 of 1620 ------------------------------
02 of 1620 ------------------------------
03 of 1620 ------------------------------
04 of 1620 ------------------------------
05 of 1620 ------------------------------
06 of 1620 ------------------------------
07 of 1620 ------------------------------
08 of 1620 ------------------------------
09 of 1620 ------------------------------
10 of 1620 ------------------------------
11 of 1620 ------------------------------
12 of 1620 ------------------------------
13 of 1620 ------------------------------
14 of 1620 ------------------------------
15 of 1620 ------------------------------
16 of 1620 ------------------------------
17 of 1620 ------------------------------
18 of 1620 ------------------------------
19 of 1620 ------------------------------
20 of 1620 ------------------------------
21 of 1620 ------------------------------
22 of 1620 ------------------------------
23 of 1620 ------------------------------
24 of 1620 -----------------------

In [39]:
discards = []
def parser_custom(TenKtext):
    
    matches = re.compile(r'(item\s(7[\.\s]|8[\.\s])|'
                         'discussion\sand\sanalysis\sof\s(consolidated\sfinancial|financial)\scondition|'
                         '(consolidated\sfinancial|financial)\sstatements\sand\ssupplementary\sdata)', re.IGNORECASE)

    matches_array = pd.DataFrame([(match.group(), match.start()) for match in matches.finditer(TenKtext)])
    print(len(matches_array), matches_array.shape, matches_array.columns)
    # Set columns in the dataframe
    matches_array.columns = ['SearchTerm', 'Start']
    print('hi', len(matches_array), matches_array.shape, matches_array.columns)
    # Get the number of rows in the dataframe
    Rows = matches_array['SearchTerm'].count()
    
    # Create a new column in 'matches_array' called 'Selection' and add adjacent 'SearchTerm' (i and i+1 rows) text concatenated
    count = 0 # Counter to help with row location and iteration
    while count < (Rows-1): # Can only iterate to the second last row
        matches_array.at[count,'Selection'] = (matches_array.iloc[count,0] + matches_array.iloc[count+1,0]).lower() # Convert to lower case
        count += 1

    # Set up 'Item 7/8 Search Pattern' regex patterns
    matches_item7 = re.compile(r'(item\s7\.discussion\s[a-z]*)')
    matches_item8 = re.compile(r'(item\s8\.(consolidated\sfinancial|financial)\s[a-z]*)')

    # Lists to store the locations of Item 7/8 Search Pattern matches
    Start_Loc = []
    End_Loc = []

    # Find and store the locations of Item 7/8 Search Pattern matches
    count = 0 # Set up counter

    while count < (Rows-1): # Can only iterate to the second last row

        # Match Item 7 Search Pattern
        if re.match(matches_item7, matches_array.at[count,'Selection']):
            # Column 1 = 'Start' column in 'matches_array'
            
            Start_Loc.append(matches_array.iloc[count,1]) # Store in list => Item 7 will be the starting location (column '1' = 'Start' column)

        # Match Item 8 Search Pattern
        if re.match(matches_item8, matches_array.at[count,'Selection']):
            End_Loc.append(matches_array.iloc[count,1])

        count += 1
    
    print(Start_Loc, End_Loc)
    input()
    if len(Start_Loc)==len(End_Loc)==0:
        return False
    if len(Start_Loc)==len(End_Loc)==1:
        print(Start_Loc, End_Loc)
    

        # Extract section of text and store in 'TenKItem7'
        TenKItem7 = TenKtext[Start_Loc[0]:End_Loc[0]]
        #print(len(TenKItem7))
    elif len(Start_Loc)!=len(End_Loc):
        TenKItem7 = TenKtext[Start_Loc[0]:End_Loc[-1]]
    else :
        print(Start_Loc, End_Loc)
        # Extract section of text and store in 'TenKItem7'
        TenKItem7 = TenKtext[Start_Loc[1]:End_Loc[1]]
        #print(len(TenKItem7))
        
    # Clean newly extracted text
    TenKItem7 = TenKItem7.strip() # Remove starting/ending white spaces
    TenKItem7 = TenKItem7.replace('\n', ' ') # Replace \n (new line) with space
    TenKItem7 = TenKItem7.replace('\r', '') # Replace \r (carriage returns-if you're on windows) with space
    TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
    TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space

    while '  ' in TenKItem7:
        TenKItem7 = TenKItem7.replace('  ', ' ') # Remove extra spaces

    # Print first 500 characters of newly extracted text
    #print(TenKItem7[:500])
    TenKItem7 = TenKItem7.replace(u'/xa0', u' ') # 
    return TenKItem7

In [46]:
from bs4 import BeautifulSoup
badfile = '../data/sec-edgar-filings/AMAT/10-K/0000006951-18-000041/full-submission.txt'
badfile_html = '../data/sec-edgar-filings/AMAT/10-K/0000006951-18-000041/filing-details.html'
badfile = '../data/sec-edgar-filings/AMAT/10-K/0000006951-16-000068/full-submission.txt'
with open(badfile) as fp:
    #soup = BeautifulSoup(fp, "lxml-xml")
    soup = BeautifulSoup(fp, "html.parser")
    print(type(soup), len(soup))
    tenK = __extract_document_text(soup)
    #tenK = filing_document.find('text').extract().text
    #tenK = soup.find('text').extract().text
    #                        with open(text_to_parse) as fp:
    #                        soup = BeautifulSoup(fp, "lxml")
     


    if tenK:
        print(len(tenK))
        TenKtext = parser_custom(tenK)
    else:
        print('nope')
    

TenKtext[:500]

<class 'bs4.BeautifulSoup'> 1
6 (6, 2) RangeIndex(start=0, stop=2, step=1)
hi 6 (6, 2) Index(['SearchTerm', 'Start'], dtype='object')
[] []

nope


TypeError: 'bool' object is not subscriptable